# Frequency Accuracy test
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf
- https://www.silabs.com/content/usergenerated/asi/cloud/attachments/siliconlabs/en/community/groups/timing/knowledge-base/jcr:content/content/primary/blog/modifying_the_feedba-K8Pv/311668.pdf

In [ ]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [ ]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [ ]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [ ]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

## Debug mode?

In [ ]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [ ]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  

In [ ]:
si.enable(True)

In [ ]:
si.enable(False)

In [ ]:
clk = si.clocks[0]
pll = si.plls[0]
ms = si.multisynths[0]

In [ ]:
clk.enable(True)

## Frequency Accuracy test

### Xtal Frequency

In [ ]:
freq_pfd = 25e6

### Minimum Clock Frequency

In [ ]:
freq_min = freq_pfd * 24 / 2048 / 128
freq_min 

### set_freq function

In [ ]:
def set_freq(freq, debug = True): 
    
    clk.set_frequency(freq)   
    freq_calculated = pll.freq / ms.divider / clk.divider
    desired_ms_divider = 25e6 * pll.divider / clk.divider / freq    
    err = ms.divider - desired_ms_divider
    ms_divider_match = ms.divider == desired_ms_divider
    
    if debug:
        print('freq:', freq)
        print()    

        print('pll.divider', pll.divider)
        print('pll.freq', pll.freq)
        print()

        print('ms.divider', ms.divider)
        print('ms.is_in_integer_mode', ms.is_in_integer_mode) 
        print('ms.is_divided_by_4', ms.is_divided_by_4)
        print('ms.freq', ms.freq)
        print()

        print('clk.divider', clk.divider)
        print('clk.freq', clk.freq)
        print('freq_calculated', freq_calculated)
        print('freq_calculated == freq' , freq_calculated == freq)
        print()  
        
    return freq, err, desired_ms_divider, ms.divider

## Set Frequency

In [ ]:
clk.enable(False)

In [ ]:
clk.enable(True)

In [ ]:
freq_min = 2289
freq_max = 225e6

In [ ]:
freq = 2289  # minimum frequency
# freq = 5.4e6
freq = 225e6 # maximum frequency
# freq = 112.4e6
# freq = 113.4e6 
# freq = 149.4e6
# freq = 150.4e6 
# freq = 97.7e6  # Radio station
freq = 90e6
freq = 25.2e6
# freq = 50.6e6

set_freq(freq)

## Sweep and test a frequency range

In [ ]:
def sweep(freq_start = freq_min, freq_end = freq_max, n_freqs = 5000 , sweep_type = 'linear'):

    if sweep_type == 'linear':
        start = freq_start
        step = (freq_end - freq_start) / n_freqs
        freqs = [(start + i * step) for i in range(n_freqs)]
    else:
        start = math.log10(freq_start)
        step = math.log10(freq_end / freq_start) / n_freqs
        freqs = [10 ** (start + i * step) for i in range(n_freqs)]
    
    return freqs

# freqs = sweep(sweep_type = 'log')
freqs = sweep(sweep_type = 'linear')

In [ ]:
validated_result = np.array([set_freq(f, debug = False) for f in freqs]).T

In [ ]:
freqs_tested = validated_result[0]
errors = validated_result[1]

In [ ]:
freqs_tested

## Frequency vs. Errors

In [ ]:
plt.plot(freqs_tested, errors);